In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from EM_dataset import EM_Data
from keras.models import Model
from keras.layers import Input
from keras.callbacks  import CSVLogger, ModelCheckpoint, TensorBoard
from keras import backend as K
from keras.optimizers import RMSprop
from keras.models import load_model
from keras.utils import multi_gpu_model

import os
import my_model

Using TensorFlow backend.


In [3]:
os.environ["CUDA_VISIBLE_DEVICES"] = "0, 1, 2, 3"  # 要放到下面的前面

In [4]:
import tensorflow as tf

config = tf.ConfigProto()
config.gpu_options.allow_growth = True
# config.gpu_options.per_process_gpu_memory_fraction = 0.7
K.set_session(tf.Session(config=config))

In [5]:
K.image_dim_ordering()

'th'

In [6]:
def soft_max_loss(y_true, y_pred):
    y_cal = y_true * y_pred
    y_cal_max = K.sum(y_cal, axis=2)
    return -K.sum(K.log(K.clip(y_cal_max, K.epsilon(),1-K.epsilon()))) \
                    /K.cast(K.prod([2, 7, 1, 256, 256]), dtype='float32')  # divided by size

In [7]:
def myacc(y_true, y_pred):
    y_true_f = K.flatten(y_true)
    y_pred_f = K.flatten(y_pred)
    return K.mean(K.equal(y_true_f, K.round(y_pred_f)))

In [8]:
model_dict = {}
models = {}

model_dict['deepEM3D'] = ['deepEM3D', my_model.timeDist_DeepEM3D_Net]
models['deepEM3D'] = 'deepEM3D'
model_name = 'deepEM3D'

# model_dict[model][1] return a function pointer for creating network model
model_name = model_dict[model_name][0]
model_create_func = model_dict[model_name][1]

In [9]:
row = 256
col = 256
depth = 7

EM_D = EM_Data(patch_shape=(row, col, depth),
              data_dir='DATA/CREMI/', data_file='A_train.h5')

loading data file from: DATA/CREMI/A_train.h5


In [10]:
class ModelMGPU(Model):
    def __init__(self, ser_model, gpus):
        pmodel = multi_gpu_model(ser_model, gpus)
        self.__dict__.update(pmodel.__dict__)
        self._smodel = ser_model

    def __getattribute__(self, attrname):
        '''Override load and save methods to be used from the serial-model. The
        serial-model holds references to the weights in the multi-gpu model.
        '''
        # return Model.__getattribute__(self, attrname)
        if 'load' in attrname or 'save' in attrname:
            return getattr(self._smodel, attrname)

        return super(ModelMGPU, self).__getattribute__(attrname)


In [11]:
inputs = Input(shape=(None, 1, row, col)) 
outputs = model_create_func(inputs)
model = Model(inputs=inputs, outputs=outputs)

In [12]:
exp = 'cremi_0'
weight_h5_file = exp + '.h5'

best_model = ModelCheckpoint(weight_h5_file, verbose=1, save_best_only=True, save_weights_only=True)  # 
csv_logger = CSVLogger('./logs/'+ exp +'.log', append=True) 
tb_logger = TensorBoard(log_dir='./logs/' + exp, write_graph=True, update_freq='epoch')

In [13]:
parallel_model = ModelMGPU(model, gpus=4)

In [14]:
optimizer = RMSprop(lr = 1e-6)

In [15]:
parallel_model.compile(optimizer = optimizer, loss = soft_max_loss, metrics = [myacc])
# parallel_model.summary()

In [17]:
history = parallel_model.fit_generator(EM_D.train_generator_aug(batch_size=16, start_slice=25), 
                              steps_per_epoch=10, epochs=500, # 1024x1024x100 / 256x256x7
                              validation_data=EM_D.valid_generator_5D(batch_size=8, slice_end=25), 
                              validation_steps=50, 
                              verbose=1, workers=8, use_multiprocessing=True,
                              callbacks = [best_model, csv_logger, tb_logger]) 
#tensorboard --logdir=/full_path_to_your_logs

Epoch 1/500
 9/10 [==========================>...] - ETA: 4s - loss: 13.4767 - myacc: 0.4370

Process ForkPoolWorker-43:
Process ForkPoolWorker-41:
Process ForkPoolWorker-44:
Process ForkPoolWorker-39:
Process ForkPoolWorker-46:
Process ForkPoolWorker-37:
Process ForkPoolWorker-45:
Traceback (most recent call last):
Traceback (most recent call last):
  File "/home/lanbowen23/anaconda3/lib/python3.6/multiprocessing/process.py", line 258, in _bootstrap
    self.run()
  File "/home/lanbowen23/anaconda3/lib/python3.6/multiprocessing/process.py", line 258, in _bootstrap
    self.run()
Process ForkPoolWorker-40:
Process ForkPoolWorker-48:
Process ForkPoolWorker-42:
  File "/home/lanbowen23/anaconda3/lib/python3.6/multiprocessing/process.py", line 93, in run
    self._target(*self._args, **self._kwargs)
  File "/home/lanbowen23/anaconda3/lib/python3.6/multiprocessing/process.py", line 93, in run
    self._target(*self._args, **self._kwargs)
  File "/home/lanbowen23/anaconda3/lib/python3.6/multiprocessing/pool.py", line 108, in worker
    task = get()
  File "/home/lanbowen23/anaconda3

KeyboardInterrupt: 

### HOW TO CONTINUE TRAIN
it turns out using multi-gpu will cause ModelCheckpoint callback to save the multi-gpu model, but what we really need is just the model on one gpu

In [11]:
if os.path.isfile(weight_h5_file):
    try:
        print('try')
        model.load_weights(weight_h5_file)
#         load_model(weight_h5_file)
    except:
        print('the model {} can not  be loaded'.format(weight_h5_file))
        pass
    

try
the model ./deepEM3D.h5 can not  be loaded


In [19]:
parallel_model.load_weights(weight_h5_file)

In [17]:
del model, parallel_model

In [18]:
%%time

parallel_model = load_model(weight_h5_file, custom_objects={'soft_max_loss': soft_max_loss,
                                                  'myacc': myacc})

CPU times: user 20.2 s, sys: 960 ms, total: 21.2 s
Wall time: 21.2 s
